In [1]:
import pandas as pd
import numpy as np

for module in pd, np:
    print(module.__name__, module.__version__)

pandas 1.2.0
numpy 1.18.5


### 1. 缺失信息的统计
缺失数据可以使用 isna 或 isnull （两个函数没有区别）来查看每个单元格是否缺失，结合 mean 可以计算出每列缺失值的比例：

In [4]:
df = pd.read_csv('learn_pandas.csv',
                 usecols = ['Grade', 'Name', 'Gender', 'Height',
                           'Weight', 'Transfer'])
df.head(10)

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N
6,Freshman,Qiang Chu,Female,162.5,52.0,N
7,Junior,Gaoqiang Qian,Female,161.9,50.0,N
8,Freshman,Changli Zhang,Female,163.0,48.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N


In [6]:
df.isna().head(10) # 布尔值返回，原表中为NaN的返回True,否则为False

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,True,False


In [7]:
df.isna().mean() # 查看缺失比例

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

如果想要同时对几个列，检索出全部为缺失或者至少有一个缺失或者没有缺失的行，可以使用 isna, notna 和 any, all 的组合。例如，对身高、体重和转系情况这3列分别进行这三种情况的检索：

In [9]:
df_sub = df[['Height', 'Weight', 'Transfer']]
df[df_sub.isna().all(1)] # 全部为缺失值

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [11]:
df[df_sub.isna().any(1)].head(3) #至少有一个缺失

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN


In [13]:
df[df_sub.notna().all(1)].head(3) # 没有缺失

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N


### 2. 缺失信息的删除
数据处理中经常需要根据缺失值的大小、比例或其他特征来进行行样本或列特征的删除， pandas 中提供了 dropna 函数来进行操作。

dropna 的主要参数为轴方向 axis （默认为0，即删除行）、删除方式 how 、删除的非缺失值个数阈值 thresh （ 非缺失值 没有达到这个数量的相应维度会被删除）、备选的删除子集 subset ，其中 how 主要有 any 和 all 两种参数可以选择。

In [15]:
# 删除身高体重至少有一个缺失的行：
one_missing_drop = df.dropna(how = 'any', subset = ['Height', 'Weight'])
one_missing_drop

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N
...,...,...,...,...,...,...
195,Junior,Xiaojuan Sun,Female,153.9,46.0,N
196,Senior,Li Zhao,Female,160.9,50.0,N
197,Senior,Chengqiang Chu,Female,153.9,45.0,N
198,Senior,Chengmei Shen,Male,175.3,71.0,N


In [16]:
# 删除超过15个缺失值的列：
res = df.dropna(1, thresh=df.shape[0]-15)
res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


 - 可以发现身高的列被删除

### 3. 缺失值的填充和插值

#### 3.1 1. 利用fillna进行填充
在 fillna 中有三个参数是常用的： value, method, limit 。其中， value 为填充值，可以是标量，也可以是索引到元素的字典映射； method 为填充方法，有用前面的元素填充 ffill 和用后面的元素填充 bfill 两种类型， limit 参数表示连续缺失值的最大填充次数。

In [17]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan],
               list('aaabcd'))
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [18]:
s.fillna(method='ffill') # 用前面的值向后填充

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [19]:
s.fillna(method='ffill', limit=1) # 连续出现的缺失，最多填充一次

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [20]:
s.fillna(s.mean()) # value为标量

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [21]:
s.fillna({'a': 100, 'd': 200}) # 通过索引映射填充的值

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

有时为了更加合理地填充，需要先进行分组后再操作。例如，根据年级进行身高的均值填充：

In [22]:
df.groupby('Grade')['Height'].transform(
                     lambda x: x.fillna(x.mean())).head()

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

#### 3.2 2. 插值函数
在关于 interpolate 函数的 文档 描述中，列举了许多插值法，包括了大量 Scipy 中的方法。由于很多插值方法涉及到比较复杂的数学知识，因此这里只讨论比较常用且简单的三类情况，即线性插值、最近邻插值和索引插值。

对于 interpolate 而言，除了插值方法（默认为 linear 线性插值）之外，有与 fillna 类似的两个常用参数，一个是控制方向的 limit_direction ，另一个是控制最大连续缺失值插值个数的 limit 。其中，限制插值的方向默认为 forward ，这与 fillna 的 method 中的 ffill 是类似的，若想要后向限制插值或者双向限制插值可以指定为 backward 或 both 。

In [24]:
s = pd.Series([np.nan, np.nan, 1,
                np.nan, np.nan, np.nan,
                2, np.nan, np.nan])
s.values

array([nan, nan,  1., nan, nan, nan,  2., nan, nan])

In [27]:
# 在默认线性插值法下分别进行 backward 和双向限制插值，同时限制最大连续条数为1：
res1 = s.interpolate(limit_direction='backward', limit=1)
print(res1.values)

print('*' * 10)
res2 = s.interpolate(limit_direction='both', limit=1)
print(res2.values)

[ nan 1.   1.    nan  nan 1.75 2.    nan  nan]
**********
[ nan 1.   1.   1.25  nan 1.75 2.   2.    nan]


In [28]:
# 第二种常见的插值是最近邻插补，即缺失值的元素和离它最近的非缺失值元素一样：
s.interpolate('nearest').values

array([nan, nan,  1.,  1.,  1.,  2.,  2., nan, nan])

 最后来介绍索引插值，即根据索引大小进行线性插值。例如，构造不等间距的索引进行演示：

In [29]:
s = pd.Series([0,np.nan,10],index=[0,1,10])
s

0      0.0
1      NaN
10    10.0
dtype: float64

In [30]:
s.interpolate() # 默认的线性插值，等价于计算中点的值

0      0.0
1      5.0
10    10.0
dtype: float64

In [31]:
s.interpolate(method='index') # 和索引有关的线性插值，计算相应索引大小对应的值

0      0.0
1      1.0
10    10.0
dtype: float64

### 4. Nullable类型
#### 1. 缺失记号及其缺陷
在 python 中的缺失值用 None 表示，该元素除了等于自己本身之外，与其他任何元素不相等：

In [32]:
print(None == None)  # True
print(None == False)  # False
print(None == [])   # False
print(None == '')  # False

True
False
False
False


另外，在 numpy 中利用 np.nan 来表示缺失值，该元素除了不和其他任何元素相等之外，和自身的比较结果也返回 False ：

In [33]:
print(np.nan == np.nan)  # False
print(np.nan == None)  # False
print(np.nan == False) # False

False
False
False


值得注意的是，虽然在对缺失序列或表格的元素进行比较操作的时候， np.nan 的对应位置会返回 False ，**但是在使用 equals 函数进行两张表或两个序列的相同性检验时，会自动跳过两侧表都是缺失值的位置，直接返回 True**

In [34]:
# 另外需要注意的是np.nan为浮点类型
type(np.nan)

float

由于 np.nan 的浮点性质，如果在一个整数的 Series 中出现缺失，那么其类型会转变为 float64 ；而如果在一个布尔类型的序列中出现缺失，那么其类型就会转为 object 而不是 bool ：

In [35]:
pd.Series([1, np.nan]).dtype

dtype('float64')

In [36]:
pd.Series([True, False, np.nan]).dtype

dtype('O')

因此，在进入 1.0.0 版本后， pandas 尝试设计了一种新的缺失类型 pd.NA 以及三种 Nullable 序列类型来应对这些缺陷，它们分别是 Int, boolean 和 string 。

#### 2. Nullable类型的性质
从字面意义上看 Nullable 就是可空的，言下之意就是序列类型不受缺失值的影响。例如，在上述三个 Nullable 类型中存储缺失值，都会转为 pandas 内置的 pd.NA ：

In [37]:
pd.Series([np.nan, 1], dtype = 'Int64') # "i"是大写的

0    <NA>
1       1
dtype: Int64

In [38]:
pd.Series([np.nan, True], dtype = 'boolean')

0    <NA>
1    True
dtype: boolean

In [39]:
pd.Series([np.nan, 'my_str'], dtype = 'string')

0      <NA>
1    my_str
dtype: string

In [40]:
pd.Series([np.nan, 0], dtype = 'Int64') + 1

0    <NA>
1       1
dtype: Int64

In [41]:
pd.Series([np.nan, 0], dtype = 'Int64') == 0

0    <NA>
1    True
dtype: boolean

In [42]:
pd.Series([np.nan, 0], dtype = 'Int64') * 0.5 # 只能是浮点

0    <NA>
1     0.0
dtype: Float64

对于 boolean 类型的序列而言，其和 bool 序列的行为主要有两点区别：

第一点是带有缺失的布尔列表无法进行索引器中的选择，而 boolean 会把缺失值看作 False ：

In [43]:
s = pd.Series(['a', 'b'])

In [44]:
s_bool = pd.Series([True, np.nan])

In [45]:
s_boolean = pd.Series([True, np.nan]).astype('boolean')

In [46]:
s[s_boolean]

0    a
dtype: object

第二点是在进行逻辑运算时， bool 类型在缺失处返回的永远是 False ，而 boolean 会根据逻辑运算是否能确定唯一结果来返回相应的值。那什么叫能否确定唯一结果呢？举个简单例子： True | pd.NA 中无论缺失值为什么值，必然返回 True ； False | pd.NA 中的结果会根据缺失值取值的不同而变化，此时返回 pd.NA ； False & pd.NA 中无论缺失值为什么值，必然返回 False 。

In [47]:
s_boolean & True

0    True
1    <NA>
dtype: boolean

In [48]:
s_boolean | True

0    True
1    True
dtype: boolean

In [49]:
~s_boolean # 取反操作同样是无法唯一地判断缺失结果

0    False
1     <NA>
dtype: boolean

#### 4.3 缺失数据的计算和分组
当调用函数 sum, prob 使用加法和乘法的时候，缺失数据等价于被分别视作0和1，即不改变原来的计算结果：

In [50]:
s = pd.Series([2,3,np.nan,4,5])
print("s.sum:" , s.sum()) # 加为0
print("s.prod:", s.prod()) # 乘为1

s.sum: 14.0
s.prod: 120.0


In [51]:
# 当使用累计函数时，会自动跳过缺失值所处的位置：
s.cumsum()

0     2.0
1     5.0
2     NaN
3     9.0
4    14.0
dtype: float64

### 练习：

Ex1：缺失值与类别的相关性检验
在数据处理中，含有过多缺失值的列往往会被删除，除非缺失情况与标签强相关。下面有一份关于二分类问题的数据集，其中 X_1, X_2 为特征变量， y 为二分类标签。

事实上，有时缺失值出现或者不出现本身就是一种特征，并且在一些场合下可能与标签的正负是相关的。关于缺失出现与否和标签的正负性，在统计学中可以利用卡方检验来断言它们是否存在相关性。按照特征缺失的正例、特征缺失的负例、特征不缺失的正例、特征不缺失的负例，可以分为四种情况，设它们分别对应的样例数为 n11,n10,n01,n00 。假若它们是不相关的，那么特征缺失中正例的理论值，就应该接近于特征缺失总数 × 总体正例的比例，即：

E11=n11≈(n11+n10)×n11+n01n11+n10+n01+n00=F11
其他的三种情况同理。现将实际值和理论值分别记作 Eij,Fij ，那么希望下面的统计量越小越好，即代表实际值接近不相关情况的理论值：

S=∑i∈{0,1}∑j∈{0,1}(Eij−Fij)2Fij
可以证明上面的统计量近似服从自由度为 1 的卡方分布，即 S∼⋅χ2(1) 。因此，可通过计算 P(χ2(1)>S) 的概率来进行相关性的判别，一般认为当此概率小于 0.05 时缺失情况与标签正负存在相关关系，即不相关条件下的理论值与实际值相差较大。

上面所说的概率即为统计学上关于 2×2 列联表检验问题的 p 值， 它可以通过 scipy.stats.chi2(S, 1) 得到。请根据上面的材料，分别对 X_1, X_2 列进行检验。

In [52]:
df = pd.read_csv('missing_chi.csv')
df.head()

,X_1,X_2,y
0,NaN,NaN,0
1,NaN,NaN,0
2,NaN,NaN,0
3,43.0,NaN,0
4,NaN,NaN,0


In [53]:
df.isna().mean()

X_1    0.855
X_2    0.894
y      0.000
dtype: float64

In [54]:
df.y.value_counts(normalize=True)

0    0.918
1    0.082
Name: y, dtype: float64

In [55]:
cat_1 = df.X_1.fillna('NaN').mask(df.X_1.notna()).fillna("NotNaN")
cat_2 = df.X_2.fillna('NaN').mask(df.X_2.notna()).fillna("NotNaN")
df_1 = pd.crosstab(cat_1, df.y, margins=True)
df_2 = pd.crosstab(cat_2, df.y, margins=True)

In [56]:
def compute_S(my_df):
     S = []
     for i in range(2):
         for j in range(2):
             E = my_df.iat[i, j]
             F = my_df.iat[i, 2]*my_df.iat[2, j]/my_df.iat[2,2]
             S.append((E-F)**2/F)
     return sum(S)

In [58]:
res1 = compute_S(df_1)
res2 = compute_S(df_2)

In [59]:
from scipy.stats import chi2

In [60]:
chi2.sf(res1, 1) # X_1检验的p值 # 不能认为相关，剔除

0.9712760884395901

In [61]:
chi2.sf(res2, 1) # X_2检验的p值 # 认为相关，保留

7.459641265637543e-166